In [101]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [102]:
cd ~/demres

/Users/zurfarosa/demres


In [103]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tools.tools import add_constant
import pylab as pl
from datetime import date, timedelta

import demres
from demres.common.constants import entry_type
from demres.common import codelists
from demres.common.helper_functions import *
from demins.functions import *

In [104]:
pd.set_option('display.max_columns', None)

## Specifiy dementia subtype

In [105]:
subtype = 'all_dementia' # options: 'alzheimers', 'vascular','all_dementia'

## Load relevant dataframe and create intercept

In [106]:
pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True)

In [107]:
pt_features.sort_values(by='matchid')

,patid,gender,yob,pracid,index_date,isCase,final dementia medcode,data_start,data_end,matchid,age_at_index_date,insomnia_window0,insomnia_window0_count,insomnia_window1,insomnia_window1_count,stroke_window0,stroke_window1,intellectual_disability,CHD_heart_failure_and_peripheral_vascular_disease_window0,CHD_heart_failure_and_peripheral_vascular_disease_window1,hypertension_window0,hypertension_window1,diabetes_window0,diabetes_window1,clin_sig_alcohol_use_window0,clin_sig_alcohol_use_window1,mental_illness_non_smi_window0,mental_illness_non_smi_window1,mental_illness_smi,sleep_apnoea_window0,sleep_apnoea_window1,current_smoker_window0,current_smoker_window1,asthma_window0,asthma_window1,COPD,CKD_window0,CKD_window1,mood_stabilisers_pdds_window0,mood_stabilisers_pdds_window1,fgas_pdds_window0,fgas_pdds_window1,sgas_pdds_window0,sgas_pdds_window1,sga_depots_pdds_window0,sga_depots_pdds_window1,fga_depots_pdds_window0,fga_depots_pdds_window1,antidepressants_pdds_window0,antidepressants_pdds_window1,consultation_count_window0,consultation_count_window1,other_sedatives_pdds_window0,other_sedatives_pdds_window1,benzo_and_z_drugs_more_than_180pdds_window0,benzo_and_z_drugs_1_to_180pdds_window0
14693,5898001,1,35,1,2008-08-12,False,0.0,1997-03-07,2013-05-14,8.0,73.0,0,0,1,2,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1672,4193,38,0,0,0,0,0,0,0,1581,2187,265,425,0,0,1,0
14718,644001,1,35,1,2008-08-12,True,1350.0,1997-03-07,2013-05-17,8.0,73.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26,35,1,0,0,0
11053,803001,0,23,1,2007-09-12,True,1350.0,1997-03-07,2012-01-19,16.0,84.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,36,79,0,0,0,0
10961,5551001,0,23,1,2007-09-12,False,0.0,1997-03-07,2012-01-04,16.0,84.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1318,96,205,0,0,1,0
14591,1064001,1,28,1,2011-03-03,True,1917.0,1997-03-07,2013-05-02,22.0,83.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,113,177,0,0,0,0
14576,1946001,1,28,1,2011-03-03,False,0.0,1997-03-07,2013-05-01,22.0,83.0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,185,237,0,0,0,0
12875,5988001,0,29,1,2007-05-24,False,0.0,1997-03-07,2012-10-26,25.0,78.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,30,0,0,0,0,0,0,0,0,1030,159,229,0,0,0,0
14518,1645001,0,29,1,2007-05-24,True,26270.0,1997-03-07,2013-04-23,25.0,78.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,92,990,29,155,3,0,0,0
14713,2136001,1,23,1,2009-03-16,True,6578.0,1997-03-07,2013-05-16,30.0,86.0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,21,99,0,0,0,0,0,0,0,0,224,287,19,0,0,1
14647,11291001,1,23,1,2009-03-16,False,0.0,1997-03-07,2013-05-09,30.0,86.0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,76,177,0,0,0,0


In [108]:
pt_features['intercept'] = 1.0

## Window 0 (i.e. exposures measured 5-10 years before index date)

### Insomnia consultations

In [109]:
training_cols_window0 = [col for col in pt_features.columns if all([col not in ['patid','insomnia_window0','benzo_and_z_drugs_1_to_180pdds_window0','benzo_and_z_drugs_more_than_180pdds_window0','index_date','matchid','data_start','data_end','final dementia medcode','isCase','yob','pracid'],'window1' not in col])]
training_cols_window0.insert(0, training_cols_window0.pop(training_cols_window0.index('insomnia_window0_count')))

In [110]:
univariate_results, multivariate_results,multivariate_summary = get_univariate_and_multivariate_results(pt_features,training_cols_window0)

Optimization terminated successfully.
         Current function value: 0.684091
         Iterations 6


#### *Univariate results*

In [111]:
univariate_results

,odds_ratio,p_value
insomnia_window0_count,1.0570,0.002
gender,1.0000,1.000
age_at_index_date,1.0000,1.000
stroke_window0,1.2732,0.000
CHD_heart_failure_and_peripheral_vascular_disease_window0,1.0944,0.015
hypertension_window0,0.9582,0.144
diabetes_window0,1.1879,0.002
mental_illness_non_smi_window0,1.4321,0.000
mental_illness_smi,1.9361,0.000
current_smoker_window0,1.0243,0.555


#### *Multivariate results*

In [112]:
multivariate_results

,odds_ratio,p_value
insomnia_window0_count,1.0178,0.324
gender,0.9638,0.238
age_at_index_date,0.9998,0.915
stroke_window0,1.1764,0.017
CHD_heart_failure_and_peripheral_vascular_disease_window0,0.9951,0.909
hypertension_window0,0.9000,0.002
diabetes_window0,1.0706,0.250
mental_illness_non_smi_window0,1.2832,0.000
mental_illness_smi,1.7505,0.000
current_smoker_window0,0.9732,0.543


In [113]:
multivariate_summary.tables[0]

Dep. Variable:,isCase,No. Observations:,18876
Model:,Logit,Df Residuals:,18856
Method:,MLE,Df Model:,19
Date:,"Sun, 07 May 2017",Pseudo R-squ.:,0.01307
Time:,21:11:33,Log-Likelihood:,-12913.
converged:,True,LL-Null:,-13084.
,,LLR p-value:,4.831e-61


### Benzo and z-drug PDDs

In [114]:
training_cols_window0 = [col for col in pt_features.columns if all([col not in ['other_sedatives_pdds_window0','fga_depots_pdds_window0','patid','insomnia_window0','insomnia_window0_count','index_date','matchid','data_start','data_end','final dementia medcode','isCase','yob','pracid'],'window1' not in col])]
training_cols_window0.insert(0, training_cols_window0.pop(training_cols_window0.index('benzo_and_z_drugs_more_than_180pdds_window0')))
training_cols_window0.insert(0, training_cols_window0.pop(training_cols_window0.index('benzo_and_z_drugs_1_to_180pdds_window0')))

In [115]:
univariate_results, multivariate_results,multivariate_summary = get_univariate_and_multivariate_results(pt_features,training_cols_window0)

Optimization terminated successfully.
         Current function value: 0.684220
         Iterations 5


#### *Univariate results*

In [116]:
univariate_results

,odds_ratio,p_value
benzo_and_z_drugs_1_to_180pdds_window0,1.1224,0.078
benzo_and_z_drugs_more_than_180pdds_window0,1.3110,0.000
gender,1.0000,1.000
age_at_index_date,1.0000,1.000
stroke_window0,1.2732,0.000
CHD_heart_failure_and_peripheral_vascular_disease_window0,1.0944,0.015
hypertension_window0,0.9582,0.144
diabetes_window0,1.1879,0.002
mental_illness_non_smi_window0,1.4321,0.000
mental_illness_smi,1.9361,0.000


#### *Multivariate results*

In [117]:
multivariate_results

,odds_ratio,p_value
benzo_and_z_drugs_1_to_180pdds_window0,0.9549,0.506
benzo_and_z_drugs_more_than_180pdds_window0,1.0413,0.599
gender,0.9644,0.247
age_at_index_date,0.9999,0.966
stroke_window0,1.1770,0.017
CHD_heart_failure_and_peripheral_vascular_disease_window0,0.9954,0.914
hypertension_window0,0.9004,0.002
diabetes_window0,1.0707,0.249
mental_illness_non_smi_window0,1.2922,0.000
mental_illness_smi,1.7413,0.000


In [118]:
multivariate_summary.tables[0]

Dep. Variable:,isCase,No. Observations:,18876
Model:,Logit,Df Residuals:,18857
Method:,MLE,Df Model:,18
Date:,"Sun, 07 May 2017",Pseudo R-squ.:,0.01288
Time:,21:12:11,Log-Likelihood:,-12915.
converged:,True,LL-Null:,-13084.
,,LLR p-value:,1.117e-60


## Window 1 (exposures measured 0-5 years before index date)

In [161]:
training_cols_window1 = [col for col in pt_features.columns if all([col not in ['patid','index_date','insomnia_window1_count','matchid','data_start','data_end','final dementia medcode','isCase','yob','pracid'],'window0' not in col])]
training_cols_window1.insert(0, training_cols_window1.pop(training_cols_window1.index('insomnia_window1')))

In [162]:
univariate_results, multivariate_results, multivariate_summary = get_univariate_and_multivariate_results(pt_features,training_cols_window1)

Optimization terminated successfully.
         Current function value: 0.664623
         Iterations 5


In [163]:
univariate_results

,odds_ratio,p_value
insomnia_window1,1.2928,0.000
gender,1.0000,1.000
age_at_index_date,1.0000,1.000
stroke_window1,1.8379,0.000
CHD_heart_failure_and_peripheral_vascular_disease_window1,0.9690,0.433
hypertension_window1,0.9526,0.161
diabetes_window1,1.1198,0.029
mental_illness_non_smi_window1,2.1245,0.000
mental_illness_smi,1.9361,0.000
current_smoker_window1,1.0655,0.107


In [164]:
multivariate_results

,odds_ratio,p_value
insomnia_window1,1.0689,0.179
gender,0.9481,0.093
age_at_index_date,0.9989,0.646
stroke_window1,1.6875,0.000
CHD_heart_failure_and_peripheral_vascular_disease_window1,0.7469,0.000
hypertension_window1,0.9032,0.010
diabetes_window1,0.9319,0.217
mental_illness_non_smi_window1,2.1525,0.000
mental_illness_smi,1.3054,0.000
current_smoker_window1,1.0470,0.298


In [165]:
multivariate_summary.tables[0]

Dep. Variable:,isCase,No. Observations:,18876
Model:,Logit,Df Residuals:,18857
Method:,MLE,Df Model:,18
Date:,"Thu, 04 May 2017",Pseudo R-squ.:,0.04115
Time:,14:34:17,Log-Likelihood:,-12545.
converged:,True,LL-Null:,-13084.
,,LLR p-value:,2.583e-217
